In [2]:
import evadb
cursor = evadb.connect(evadb_dir=("./eva_db")).cursor()

In [3]:
cursor.query("""
DROP TABLE IF EXISTS mouses
""").df()
cursor.query("""
CREATE TABLE mouses (
            area INTEGER,
            price INTEGER
            )"""
).df()
for area, price in zip([30, 35, 37, 40, 45, 50, 50, 60, 65, 70] * 10,
                       [300, 350, 370, 400, 450, 450, 500, 600, 650, 700] * 10):
    cursor.query("""
    INSERT INTO mouses (area, price) VALUES ({}, {})
    """.format(area, price)).df()
cursor.query("""
SELECT * FROM mouses
""").df()


,_row_id,area,price
0,1,30,300
1,2,35,350
2,3,37,370
3,4,40,400
4,5,45,450
...,...,...,...
95,96,50,450
96,97,50,500
97,98,60,600
98,99,65,650


In [4]:
# example
# CREATE OR REPLACE FUNCTION PredictHouseRent FROM
# ( SELECT number_of_rooms, number_of_bathrooms, days_on_market, rental_price FROM HomeRentals )
# TYPE Sklearn
# PREDICT 'rental_price';

cursor.query("""
CREATE OR REPLACE FUNCTION PredictMouseRent FROM
( SELECT area, price FROM mouses )
TYPE Modal
PREDICT 'price';
""").df()


,0
0,Function PredictMouseRent overwritten.


In [5]:
cursor.query("""
SELECT area, price, PredictMouseRent(area, price) FROM mouses
""").df()

args ()
kwargs {'predict': 'price', 'target_col': 'price', 'col_mode': '450'}
foward on n_frames =  100


,area,price,price
0,30,300,450
1,35,350,450
2,37,370,450
3,40,400,450
4,45,450,450
...,...,...,...
95,50,450,450
96,50,500,450
97,60,600,450
98,65,650,450


In [33]:
query = """
CREATE OR REPLACE FUNCTION DownloadURL IMPL './dummy_fn.py'
"""
cursor.query(query).df()
# create a table of urls
cursor.query("""
DROP TABLE IF EXISTS urls
""").df()
cursor.query("""
CREATE TABLE urls (
            url TEXT(0)
            )"""
).df()

urls = [
    "https://www.wikipedia.org/",
    "https://www.github.com/",
    "https://www.stackoverflow.com/",
    "https://www.medium.com/",
    "https://www.nasa.gov/",
    "https://www.nature.com/",
    "https://www.bbc.com/news",
    "https://www.cnn.com/",
    "https://www.nytimes.com/",
    "https://www.theguardian.com/international",
    "https://www.forbes.com/",
    "https://www.nationalgeographic.com/"
]
for url in urls:
    cursor.query("""
    INSERT INTO urls (url) VALUES ('{}')
    """.format(url)).df()
# show
cursor.query("""
SELECT * FROM urls
""").df()
# apply DownloadURL function
cursor.query("""
SELECT url, DownloadURL(url)  FROM urls
""").df()

,url,html
0,https://www.wikipedia.org/,"<!DOCTYPE html>\n<html lang=""en"" class=""no-js""..."
1,https://www.github.com/,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" ..."
2,https://www.stackoverflow.com/,"<!DOCTYPE html>\n\n\n <html class=""html__re..."
3,https://www.medium.com/,"<!doctype html><html lang=""en""><head><title da..."
4,https://www.nasa.gov/,"<!doctype html>\n<html lang=""en-US"" prefix=""og..."
5,https://www.nature.com/,"<!DOCTYPE html>\n<html lang=""en"" class=""grade-..."
6,https://www.bbc.com/news,"<!DOCTYPE html><html><head><meta charSet=""utf-..."
7,https://www.cnn.com/,"<!DOCTYPE html>\n<html lang=""en"" data-uri=""c..."
8,https://www.nytimes.com/,"<!DOCTYPE html>\n<html lang=""en"" class="" nytap..."
9,https://www.theguardian.com/international,"<!doctype html>\n <html lang=""en"">\n ..."


In [36]:
# create a dummy table of search words 
cursor.query("""
DROP TABLE IF EXISTS searches
""").df()
cursor.query("""
CREATE TABLE searches (
            search TEXT(0)
            )"""
).df()
# insert a few entries
searches = [
    "python",
    "c++",
    "java",
]
# insert
for search in searches:
    cursor.query("""
    INSERT INTO searches (search) VALUES ('{}')
    """.format(search)).df()
# show
cursor.query("""
SELECT * FROM searches
""").df()
# register the bing_search function at ./bing_search.py
cursor.query("""
CREATE OR REPLACE FUNCTION BingSearch IMPL './bing_search.py'
""").df()
# apply BingSearch
cursor.query("""
SELECT search, BingSearch(search)  FROM searches
""").df()

,search,links
0,python,"[https://www.python.org/, https://www.w3school..."
1,c++,"[https://en.wikipedia.org/wiki/C++, https://si..."
2,java,"[https://www.java.com/, https://www.oracle.com..."


In [40]:
# register extract_soup function from './extract_soup.py'
cursor.query("""
CREATE OR REPLACE FUNCTION ExtractSoup IMPL './extract_soup.py'
""").df()
# apply ExtractSoup to a dummy html file
dummy_html = """
<html>
<head>
<title>Page Title</title>
</head>
<body>
<h1>This is a Heading</h1>
<p>This is a paragraph.</p>
</body>
</html>
"""
cursor.query("""
SELECT ExtractSoup('{}')
""".format(dummy_html)).df()

,text
0,Page Title\nThis is a Heading\nThis is a parag...


In [50]:
# create a dummy table of questions: what is 1 + 1, what is 2 + 2, etc.
cursor.query(""" DROP TABLE IF EXISTS questions """).df()
cursor.query(""" CREATE TABLE questions ( question TEXT(0) )""").df()
# insert a few entries
questions = [ "what is 1 + 1", "what is 2 + 2", 
                "what is 3 + 3", "what is 4 + 4", 
                "what is 5 + 5", "what is 6 + 6", 
                "what is 7 + 7", "what is 8 + 8", 
                "what is 9 + 9", "what is 10 + 10",
                "what is 11 + 11", "what is 12 + 12",
                "what is 13 + 13", "what is 14 + 14",
                "what is 15 + 15", "what is 16 + 16",]
for question in questions:
    cursor.query(""" INSERT INTO questions (question) VALUES ('{}') """.format(question)).df()
# show
cursor.query(""" SELECT * FROM questions """).df()


,_row_id,question
0,1,what is 1 + 1
1,2,what is 2 + 2
2,3,what is 3 + 3
3,4,what is 4 + 4
4,5,what is 5 + 5
5,6,what is 6 + 6
6,7,what is 7 + 7
7,8,what is 8 + 8
8,9,what is 9 + 9
9,10,what is 10 + 10


In [51]:
cursor.query(""" CREATE OR REPLACE FUNCTION gpt_search IMPL './gpt.py' """).df()
cursor.query(""" SELECT question, gpt_search(question) FROM questions """).df()
# 5s for concurrent processing of gpt_search

,question,reply
0,what is 1 + 1,1 + 1 equals 2.
1,what is 2 + 2,2 + 2 equals 4.
2,what is 3 + 3,3 + 3 equals 6.
3,what is 4 + 4,4 + 4 equals 8.
4,what is 5 + 5,5 + 5 equals 10.
5,what is 6 + 6,The sum of 6 + 6 is 12.
6,what is 7 + 7,7 + 7 equals 14.
7,what is 8 + 8,The sum of 8 + 8 is 16.
8,what is 9 + 9,The sum of 9 + 9 is 18.
9,what is 10 + 10,10 + 10 equals 20.
